In [0]:
CREATE LIVE TABLE gold_top_games_daily
AS
SELECT
  ingestion_date,
  game_name,
  COUNT(DISTINCT username) AS streamer_count,
  SUM(viewer_count) AS total_viewers,
  ROUND(AVG(viewers_per_minute), 2) AS avg_viewers_per_minute
FROM LIVE.silver_streams
WHERE game_name IS NOT NULL AND viewer_count > 0
GROUP BY ingestion_date, game_name;

In [0]:
CREATE LIVE TABLE gold_streamer_performance
AS
SELECT
  username,
  display_name,
  MAX(account_age_years) AS account_age_years,
  MAX(followers_count) AS total_followers,
  COUNT(*) AS live_appearances,
  ROUND(AVG(viewer_count), 2) AS avg_viewers,
  ROUND(AVG(viewers_per_minute), 2) AS avg_viewers_per_minute,
  MAX(ingestion_time) AS last_seen
FROM LIVE.silver_joined_data
GROUP BY username, display_name;

In [0]:
CREATE LIVE TABLE gold_hourly_viewership_trend
AS
SELECT
  ingestion_date,
  HOUR(ingestion_time) AS hour,
  game_name,
  COUNT(*) AS streams,
  SUM(viewer_count) AS total_viewers
FROM LIVE.silver_streams
GROUP BY ingestion_date, HOUR(ingestion_time), game_name;

In [0]:
CREATE LIVE TABLE gold_streamer_followers_history
COMMENT "Hourly snapshot of streamer followers for trend analysis"
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  user_id,
  username,
  followers_count,
  data_collection_time,
  collection_date
FROM LIVE.silver_streamers;

In [0]:
CREATE LIVE TABLE gold_streamer_follower_deltas
COMMENT "Hourly follower count changes per streamer"
TBLPROPERTIES ("quality" = "gold")
AS
WITH calculated_deltas AS (
  SELECT
    user_id,
    username,
    followers_count,
    data_collection_time,
    collection_date,
    followers_count - LAG(followers_count) OVER (
      PARTITION BY user_id ORDER BY data_collection_time
    ) AS followers_delta,
    LAG(data_collection_time) OVER (
      PARTITION BY user_id ORDER BY data_collection_time
    ) AS previous_collection_time
  FROM LIVE.gold_streamer_followers_history
)
SELECT
  *
FROM calculated_deltas
WHERE followers_delta IS NOT NULL;

In [0]:
CREATE LIVE TABLE gold_avg_daily_follower_growth
COMMENT "Average daily follower growth per streamer"
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  user_id,
  username,
  MIN(followers_count) AS min_followers,
  MAX(followers_count) AS max_followers,
  MIN(data_collection_time) AS first_snapshot,
  MAX(data_collection_time) AS last_snapshot,
  DATEDIFF(MAX(data_collection_time), MIN(data_collection_time)) AS days_observed,
  CASE 
    WHEN DATEDIFF(MAX(data_collection_time), MIN(data_collection_time)) > 0 
    THEN ROUND(
      (MAX(followers_count) - MIN(followers_count)) / 
      DATEDIFF(MAX(data_collection_time), MIN(data_collection_time)), 2)
    ELSE NULL
  END AS avg_daily_follower_growth
FROM LIVE.gold_streamer_followers_history
GROUP BY user_id, username;